# Word to Vector:

In [0]:
import gensim
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import re
import pandas as pd
import nltk
import numpy as np
import pickle as pkl

In [6]:
### reading in master df
nltk.download('punkt')
nltk.download('stopwords')
master_text = pd.read_csv('/content/drive/My Drive/covid 19 exploration/df/master_text.csv')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Training Word2Vec Model:

In [0]:
def clean(sentence):
    tokens = word_tokenize(sentence)
    # convert to lower case
    tokens = [w.lower() for w in tokens]

    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))

    # remove punctuation from each word
    # sub function susbstitutes punctuation with nothing
    stripped = [re_punc.sub('', w) for w in tokens]

    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]

    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

In [0]:
# Applying the aboe function on the corpus of documents
# Creating a list of list of tokens for the set of first 100 reviews (calling it corpus)
reviews_list=master_text.main_text.to_list()
corpus = []
for doc in reviews_list:
    corpus.append(clean(doc))
%cd "/content/drive/My Drive/covid 19 exploration/word_2_vec"
pkl.dump(corpus, open('corpus.pkl', 'wb'))

In [113]:
from gensim.models import Word2Vec

%cd "/content/drive/My Drive/covid 19 exploration/word_2_vec"
model = Word2Vec(corpus, min_count=2, workers = 3)
model.save('word2vec.model')

/content/drive/My Drive/covid 19 exploration/word_2_vec


## Creating a Word2Vec Search Engine:

In [23]:
%cd "/content/drive/My Drive/covid 19 exploration/word_2_vec"
pkl.dump(corpus, open('corpus.pkl', 'wb'))

/content/drive/My Drive/covid 19 exploration/word_2_vec


In [88]:
from gensim.models import Word2Vec
import pickle as pkl
%cd "/content/drive/My Drive/covid 19 exploration/word_2_vec"
model_2 = Word2Vec.load("word2vec.model")
corpus=pkl.load(open('corpus.pkl', 'rb'))
vocab = set(model_2.wv.vocab.keys())

/content/drive/My Drive/covid 19 exploration/word_2_vec


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [92]:
import sys
search = input('Please Enter your search criteria: ')

token_search=clean(search)
token_search = set(word for word in token_search if word in vocab)

smilarity=[]

df = master_text
print('Cleaned input Sentence')

counter = 0
length = len(corpus)
skiped = 0
for i, tokens in enumerate(corpus):
  # tokens = set(tokens)
  percent = 100*(i+1)/length
  tokens = [word for word in tokens if word in vocab]
  try: 
    smilarity.append(model_2.n_similarity(token_search, tokens))
  except:
    smilarity.append(np.nan)
    skiped += 1
  if (i+1) % 100 == 0:
    sys.stdout.write(f'\riteration={i:2d} status={percent:0.2f}% skiped={skiped}')



df['Similarity'] = smilarity

df.sort_values('Similarity', ascending = False)

Please Enter your search criteria: doctor
Cleaned input Sentence


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


iteration=29299 status=99.95% skiped=17

,Unnamed: 0,paper_id,papepr_title,abstract,main_text,file_path,Similarity
21423,21423,ac20ef2db6e48e153c04845d8cb18f179e91e90a,SARS e My personal battle,NaN,Summary It isn't every day that a doctor beco...,custom_licens,0.712052
11315,11315,13ef940f453b41f002b5525a2a575a05da87b1d6,NaN,NaN,"A Call to Respond, Not to React Kathy Robinso...",custom_licens,0.680221
22656,22656,bea6c6081961230c8d79e836f9daf2dcb54f6e7c,NaN,NaN,"Tanabe P, Steinmann R, Kippenhan M, Stehman C...",custom_licens,0.677022
24110,24110,d5449e74dec7d21e090e159e85e70d3c510d66af,Journal Pre-proof What are we doing in the der...,NaN,"In late December 2019, several cases with une...",custom_licens,0.673645
20685,20685,a0af5eb87954bafb165b50feb456353b1f0cb345,Office of Steering Committee,Introduction: A large general hospital was su...,Severe acute respiratory distress syndrome (S...,custom_licens,0.667047
...,...,...,...,...,...,...,...
24077,24077,d4cabddf62ffb457d267c77bf7aa0b27e69027dd,NaN,NaN,"49, 84, 99, 205, 223, 244, 259, 281, 321, 32...",custom_licens,NaN
25907,25907,f09c5c37a3dd5e55910f33acd45db29ebfdd2171,NaN,NaN,"[271] [272] [273] 26, [457] [458] [566] [567...",custom_licens,NaN
26213,26213,f508e125b0592b51a7ca2edf3ebcde80941640d4,NaN,A Alkalinization Opt im izing management of t...,(2),custom_licens,NaN
27861,27861,617197cc751a9208cb0af1b4e31baeddc8d2e985,"""The Duty to Prevent"" during an epidemic situa...",NaN,"역학은 질병예방과 건강증진을 목적으로 관련한 요인들을 규명하고, 이들 근거들을 지...",noncomm_use_subse,NaN
